 ## Real-Time Latency Monitoring

### Objective: Trigger alerts when latency exceeds a critical threshold. This will store high-latency events in Azure Synapse.<br>

<br>Processing Steps:<br>

If latency > 500ms, trigger an Azure Function to send an alert via email/SMS.

In [ ]:
SELECT type, tstamp, latency
INTO [bubble-real]
FROM [bubble-eventtest]
WHERE latency > 500000.0;


SlidingWindow(Second, 10) → Collects data continuously over the last 10 seconds.<br>
Updates every second (so it is always monitoring in real time).<br>
Uses TIMESTAMP BY tstamp to ensure correct event ordering.<br>

In [ ]:
SELECT 
type, 
System.Timestamp AS event_time, 
AVG(latency) AS avg_latency
INTO [bubble-real]
FROM [bubble-eventtest] TIMESTAMP BY tstamp
GROUP BY type, SlidingWindow(Second, 10);

### Calculate Average Latency Per Event Type
This will aggregate latency every 10 seconds.



In [ ]:
SELECT type, AVG(latency) AS avg_latency
INTO [bubble-real]
FROM [bubble-eventtest] TIMESTAMP BY tstamp
GROUP BY type, TumblingWindow(Second, 10);

## Event Rate Monitoring (Throughput Analytics)
### Objective: Monitor how frequently different types of events are generated.
<br>Processing Steps:<br>

Count the number of MAC, RLC, PDCP, GTP messages per minute.<br>
Identify if there’s a spike or drop in event generation.

In [ ]:
SELECT type, COUNT(*) AS event_count
INTO [bubble-real]
FROM [bubble-eventtest] TIMESTAMP BY tstamp
GROUP BY type, TumblingWindow(Second, 30);

### Detect Packet Loss (Missing Events)

This will detect if fewer than 10 events arrive in 10 seconds.

In [ ]:
WITH EventCounts AS (
    SELECT type, COUNT(*) AS event_count
    FROM [bubble-eventtest] TIMESTAMP BY tstamp
    GROUP BY type, TumblingWindow(Second, 10)
)
SELECT type, event_count
INTO [bubble-real]
FROM EventCounts
WHERE event_count < 10;